In [49]:
import pandas as pd
import numpy as np
import scipy
import math

In [1]:
def GoalSeek(fun,goal,x0,fTol=0.0001,MaxIter=1000):
    # Goal Seek function of Excel
    #   via use of Line Search and Bisection Methods

    # Inputs
    #   fun     : Function to be evaluated
    #   goal    : Expected result/output
    #   x0      : Initial estimate/Starting point

    # Initial check
    if fun(x0)==goal:
        print('Exact solution found')
        return x0

    # Line Search Method
    step_sizes=np.logspace(-1,4,6)
    scopes=np.logspace(1,5,5)

    vFun=np.vectorize(fun)

    for scope in scopes:
        break_nested=False
        for step_size in step_sizes:

            cApos=np.linspace(x0,x0+step_size*scope,int(scope))
            cAneg=np.linspace(x0,x0-step_size*scope,int(scope))

            cA=np.concatenate((cAneg[::-1],cApos[1:]),axis=0)

            fA=vFun(cA)-goal

            if np.any(np.diff(np.sign(fA))):

                index_lb=np.nonzero(np.diff(np.sign(fA)))

                if len(index_lb[0])==1:

                    index_ub=index_lb+np.array([1])

                    x_lb=np.asscalar(np.array(cA)[index_lb][0])
                    x_ub=np.asscalar(np.array(cA)[index_ub][0])
                    break_nested=True
                    break
                else: # Two or more roots possible

                    index_ub=index_lb+np.array([1])

                    print('Other solution possible at around, x0 = ', np.array(cA)[index_lb[0][1]])

                    x_lb=np.asscalar(np.array(cA)[index_lb[0][0]])
                    x_ub=np.asscalar(np.array(cA)[index_ub[0][0]])
                    break_nested=True
                    break

        if break_nested:
            break
    if not x_lb or not x_ub:
        print('No Solution Found')
        return

    # Bisection Method
    iter_num=0
    error=10

    while iter_num<MaxIter and fTol<error:
        
        x_m=(x_lb+x_ub)/2
        f_m=fun(x_m)-goal

        error=abs(f_m)

        if (fun(x_lb)-goal)*(f_m)<0:
            x_ub=x_m
        elif (fun(x_ub)-goal)*(f_m)<0:
            x_lb=x_m
        elif f_m==0:
            print('Exact spolution found')
            return x_m
        else:
            print('Failure in Bisection Method')
        
        iter_num+=1

    return x_m

In [50]:
f_h=5
dis_rate=11.96
cur_net_inc=383270.00
lp_cf = 100
cur_cash_flow = (cur_net_inc + (lp_cf/100))
ter_gro_rate = 6.00
int_gro_rate = 18.2252750
cur_share_price = 3064.90
no_share = 3659.05

In [51]:
fut_cas_flow = [cur_cash_flow,]
cas_flow = (cur_cash_flow * (1+(int_gro_rate/100)))
fut_cas_flow.append(round(cas_flow,2))

for i in range(1,f_h):
    cas_flow1=(fut_cas_flow[-1] * (1+(int_gro_rate/100)) )
    fut_cas_flow.append(round(cas_flow1,2))
print(fut_cas_flow)

[383271.0, 453123.19, 535706.14, 633340.06, 748768.03, 885233.06]


In [60]:
df=pd.DataFrame()
df['year']=np.arange(0,(f_h+1))
df["fut_cas_flow"]= fut_cas_flow
k=[y for y in range(0,(f_h+1))]
df['pv_fut_cf']= [((i)/pow((1+(dis_rate/100)),k)) for i in (df['fut_cas_flow'])]

TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'list'

In [59]:
[((i)/pow((1+(dis_rate/100)),k)) for i in (df['fut_cas_flow'])]

TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'list'

In [34]:
df

,year,fut_cas_flow,pv_fut_cf
0,0,383271.00,8.931761e-01
1,1,453141.30,4.047350e+05
2,2,535748.96,2.563656e+11
3,3,633416.00,2.269887e+17
4,4,748887.74,2.809338e+23
5,5,885409.98,4.860256e+29


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          6 non-null      int32  
 1   fut_cas_flow  6 non-null      float64
dtypes: float64(1), int32(1)
memory usage: 200.0 bytes
